<a href="https://colab.research.google.com/github/rbolekr/ipynb/blob/main/Gymi_3_Info_Generator_f%C3%BCr_mathematische_%C3%9Cbungen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description

## Technical Details

This project uses several external packages:
- [SymPy](https://www.sympy.org/en/index.html) for symbolic manipulation of the various functions used by the question generators.
- [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) for the user interface.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

class Interface:
    def __init__(self, question_label_value='', answer_label_value='', check_button_click_handler=None):
        self.question_label_value = question_label_value
        self.answer_label_value = answer_label_value
        self.check_button_click_handler = check_button_click_handler

        self.tab = None
        self.answer_text = None
        self.check_button = None

        self.create_interface()

    def create_interface(self):
        # The interface will be composed of two tabs.
        self.tab = widgets.Tab()        

        # Tab 1: 'Question'
        question_label = widgets.HTML(
            value=f'<br><h1><center>{self.question_label_value}</center></h1><br><br>',
            placeholder='',
            description='',
            layout=widgets.Layout(width='75%')
        )

        self.answer_text = widgets.Text(
            value='',
            description='Answer:',
            layout=widgets.Layout(width='75%')
        )

        # This is a nice-to-have: Initially the 'Check' button is disable and it's only enabled when the
        # contents of the 'Answer' text are not empty. To handle that we need to observe for change events.
        self.answer_text.observe(self.on_text_change, names='value')

        self.check_button = widgets.Button(
            description='Check',
            button_style='primary',
            layout=widgets.Layout(width='75%'),
            disabled=True
        )

        # Important: By default the handler will be notified of which button was clicked which is not useful.
        # We want the text of a different UI element so we have to intercept the call and provide it ourselves.
        self.check_button.on_click(self.check_button_click_handler)

        question_tab_box = widgets.VBox(
            [question_label, self.answer_text, self.check_button],
            layout=widgets.Layout(width='33%')
        )

        # Tab 2: 'Answer'
        answer_label = widgets.HTML(
            value=f'<br><h1><center>The correct answer is: {self.answer_label_value}</center></h1><br><br>',
            placeholder='',
            description='',
            layout=widgets.Layout(width='75%')
        )

        answer_tab_box = widgets.VBox(
            [answer_label],
            layout=widgets.Layout(width='33%')
        )

        self.tab.children = [question_tab_box, answer_tab_box]
        self.tab.set_title(0, 'Question')
        self.tab.set_title(1, 'Answer')

    def handle_button_click(self, handler):
      return lambda b: handler(self.answer_text.value)

    def on_text_change(self, change):
        self.check_button.disabled = len(change['new'].strip()) == 0

    def display(self):
        display(self.tab)

# Generator 1: Multiplication of two numbers in the range of 1 to 10.

This first generator is meant for the youngest math learners. For example, my 5 year old brother Luca.

In [ ]:
import random
from sympy import *

# Variables and function (simple multiplication) definition:
a, b = symbols('a, b')
f = Mul(a, b)

# Two random concrete values:
a_concrete = random.randint(1, 10)
b_concrete = random.randint(1, 10)
solution = int(f.evalf(subs={a:a_concrete, b:b_concrete}))

def click_handler(button):
    value = interface.answer_text.value.strip()
    try:
        answer_int_value = int(value)
        if answer_int_value == solution:
            print(f'Correct! Please reload for another question.')
        else:
            print(f'{answer_int_value} is incorrect. Try again or peak at the other tab!')
    except ValueError:
        print(f'"{value}" does not seem like a number. Try again or peak at the other tab!')

interface = Interface(
    question_label_value=f'What is {a_concrete} x {b_concrete}?',
    answer_label_value=f'{solution}',
    check_button_click_handler=click_handler
)

interface.display()